In [ ]:
import os, sys

dirpath = os.path.join( '/'.join(os.getcwd().split("/")[:-5]), 'src')
sys.path.insert(0, dirpath)

# **Bias measuring and mitigation in classification tasks**

With the increasing use of machine learning models in different areas, it has become important to address the bias problem in these models. This issue can appear in different aspects such as racial, gender or socioeconomic biases leading to unfair outcomes in decision-making processes, for instance, in classification tasks, where models are trained to classify data into different categories.

There are various techniques to measure bias in classification models such as Equalized Odds, Demographic Parity or Opportunity Equality for instance, which quantify the differences in the model's performance across different subgroups based on sensitive attributes such as gender or race.

Once bias is detected, we can employ different techniques to mitigate it. These methods can be grouped into three categories: Pre-processing, in-processing and post-processing methods. Pre-processing techniques are used to adjust the training data to remove bias, while in-processing methods are applied to build robust models against bias. Finally, post-processing techniques are used to adjust the model's predictions to remove bias.

Through this tutorial, we pretend to present you tools which can be easily applied to measure and mitigate the presence of bias in classification models.

We will follow the traditional outline for this tutorial:

1. Data loading and packages installation
2. Dataset preprocessing
3. Data analysis
4. Model training
5. Bias measuring
6. Bias mitigation
7. Results comparison

## **1. Data loading and packages installation**

First of all, we need to import the required packages to perform our bias analysis and mitigation. You will need to have the `holisticai` package installed on your system, remember that you can install it by running: 
```bash
!pip install holisticai[all]
```

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(0)
import warnings
warnings.filterwarnings("ignore")

## **2. Dataset preprocessing**

The dataset that we will use is the "Adult" dataset from the UCI Machine Learning Repository, this is a publicly available dataset that contains information about age, education, marital status, race and gender of individuals from the United States. The objective is to predict whether an individual's income will be above or below $50K per year.

Source: [UCI ML repository](https://archive.ics.uci.edu/ml/datasets/adult)

In [ ]:
from holisticai.datasets import load_dataset

dataset = load_dataset('adult')
dataset

Now that we have a clean dataset we can start defining the training and testing sets.

## **3. Data analysis**

Since this function already return the protected groups, we can start with the data analysis step. In this step, we will analyze the distribution of the data of the protected groups by using the `group_pie_plot` function. This function will plot the distribution of the data of the protected groups in a pie chart.

In [ ]:
from holisticai.plots.bias import group_pie_plot, frequency_plot
import matplotlib.pyplot as plt

group_a = dataset['p_attr']['group_a']
group_b = dataset['p_attr']['group_b']
y = dataset['y']

fig,axs = plt.subplots(1, 2, figsize=(15,5))
group_pie_plot(group_a, ax=axs[0], title='Group Pie Plot')
frequency_plot(group_a, dataset['y'], ax=axs[1], title='Frequency Plot')

The previous graphs show us that the proportion of male (labelled as *False*) is high compared to the female.

Let's plot the columns with the highest correlations with respect to the target variable. To do this, we will use the `correlation_matrix_plot` function. 

In [ ]:
from holisticai.plots.bias import correlation_matrix_plot

df = pd.concat([dataset['x'], dataset['y']], axis=1)

correlation_matrix_plot(df, target_feature='y', n_features=10)

Now that we have our protected groups, we will define the input/output sets.

In [ ]:
dataset_split = dataset.train_test_split(test_size=0.3)
dataset_split

In [ ]:
# train a model, do not forget to standard scale data
train = dataset_split['train']
test = dataset_split['test']

group_a_test = test['p_attr']['group_a']
group_b_test = test['p_attr']['group_b']
group_a_train = train['p_attr']['group_a']
group_b_train = train['p_attr']['group_b']

## **4. Model training**

Once we have defined the input and output sets, we can train a model as usual. Since the protected groups were separated from the dataset previously we do not need to take care of that in this opportunity, but do not forget to separate the protected attributes from the dataset, so that the model does not have any influence from these attributes in its training process. For the training process, we will use a traditional pipeline, we will fit and re-scale the training data, and then we will use the data to train a "Logistic regression" model and once the model has been trained, we can use its predictions to calculate the fairness metrics of the it.

In [ ]:
# sklearn imports
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from holisticai.pipeline import Pipeline

# train a model, do not forget to standard scale data
train = dataset_split['train']
test = dataset_split['test']

scaler = StandardScaler()
X_train_t = scaler.fit_transform(train['x'])
model = LogisticRegression(random_state=42, max_iter=500)
model.fit(X_train_t, train['y'])
X_test_t = scaler.transform(test['x'])
y_pred = model.predict(X_test_t)

## **5. Bias measuring**

The fairness of the model can be calculated using the predictions of the model and the protected groups defined previously. The `holisticai` contains a module that calculates a set of metrics useful in evaluating the fairness of algorithmic decisions. For our case, we will use the `classification_bias_metrics6` function which allows us to select which metrics we want to calculate, if `equal_outcome`, `equal_opportunity` or `both`, where equal_outcome shows how disadvantaged groups are treated by the model and equal_opportunity shows if all the groups have the same opportunities.

In [ ]:
from holisticai.bias.metrics import classification_bias_metrics

In [ ]:
df = classification_bias_metrics(
    group_a_test,
    group_b_test,
    y_pred,
    test['y'],
    metric_type='both'
)
y_baseline = y_pred.copy()
df_baseline=df.copy()
df_baseline

For example:

*   **Statistical parity:** Computes the difference in success rates between the protected groups. Values below 0 are considered unfair towards `group_a` while values above 0 are considered unfair towards `group_b`, the range (-0.1, 0.1) is considered acceptable.
*   **Disparate Impact:** Shows the ratio of success rates between the protected groups for a certain quantile. Values below 1 are unfair towards `group_a`. Values above 1 are unfair towards `group_b`. The range (0.8, 1.2) is considered acceptable.
*   **Four Fifths:** Computes the ratio of success rates between the protected groups. Values below 1 are considered unfair while a range between (0.8, 1) is considered acceptable.
*   **Cohen D:** Computes the normalised statistical parity between the protected groups. Values below 0 are considered unfair towards `group_a` while values above 0 are considered unfair towards `group_b`.
*   **Equality of opportunity difference:** Computes the difference in true positive rates between the protected groups. Values below 0 are considered unfair towards `group_a` while values above 0 are considered unfair towards `group_b`.
*   **False positive rate difference:** Computes the difference in false positive rates between the protected groups, negative values indicating bias against `group_a` while positive values indicating bias against `group_b`.
*   **Average Odds Difference:** Computes the difference in average odds between the protected groups, negative values indicating bias against `group_a` while positive values indicating bias against `group_b`, a range between (-0.1, 0.1) is considered acceptable.
*   **Accuracy Difference:** Computes the difference in accuracy of predictions for the protected groups, positive values show bias against `group_b` while negative values show bias against `group_a`.


Source: [*HolisticAI docs*](https://holisticai.readthedocs.io/en/latest/metrics.html#binary-classification)

## **6. Bias mitigation**
### **Adversarial debiasing: Inprocessing method**

Now that we could observe that the model metrics are far away from the desired values, we need to apply some kind of strategy to mitigate the bias present in the model.

Exists different kinds of strategies, and the literature has divided them into three categories: *Pre-processing*, *in-processing* and *post-processing* methods. The `holisticai` library possesses different algorithms from these categories for bias mitigation. An interesting feature is that all of them are compatible with the `Scikit-learn` package, so that, if you are familiar with this package, you will not have problems using the library. As you will see later, its implementation can be done following the traditional way, or by applying the pipeline. 

For this opportunity, we will implement the "Adversarial debiasing" method which is an in-processing technique that learns a classifier to maximize prediction accuracy and simultaneously reduce an adversary's ability to determine the protected attribute from the predictions.

### **Traditional implementation**

In [ ]:
from holisticai.bias.mitigation import AdversarialDebiasing
np.random.seed(10)

In [ ]:
inprocessing_model = AdversarialDebiasing(
        features_dim=train['x'].shape[1],
        batch_size=32,
        hidden_size=64,
        adversary_loss_weight=0.1,
        verbose=1,
        use_debias=True,
        seed=42,
    ).transform_estimator()

scaler = StandardScaler()
X_train = scaler.fit_transform(train['x'])

inprocessing_model.fit(X_train, train['y'], group_a_train, group_b_train)


X_test = scaler.transform(test['x'])
y_pred = inprocessing_model.predict(X_test)

df = classification_bias_metrics(
    group_a_test,
    group_b_test,
    y_pred,
    test['y'],
    metric_type='both'
)
df_adv_deb = df.copy()
df_adv_deb

### **Pipeline implementation**

Now that we could see how this method is implemented traditionally, let's try implementing it by using the Scikit-learn Pipeline! Notice that we shouldn't have variations in the metrics for the previous implementation.

In [ ]:
inprocessing_model = AdversarialDebiasing(
        features_dim=X.shape[1],
        batch_size=32,
        hidden_size=64,
        adversary_loss_weight=0.1,
        verbose=1,
        use_debias=True,
        seed=42,
    ).transform_estimator()

pipeline = Pipeline(
    steps=[
        ('scalar', StandardScaler()),
        ("bm_inprocessing", inprocessing_model),
    ]
)

X, y, group_a, group_b = train_data
fit_params = {
    "bm__group_a": group_a, 
    "bm__group_b": group_b
}

pipeline.fit(X, y, **fit_params)

X, y, group_a, group_b = test_data
predict_params = {
    "bm__group_a": group_a,
    "bm__group_b": group_b,
}
y_pred = pipeline.predict(X, **predict_params)
df = classification_bias_metrics(
    group_a,
    group_b,
    y_pred,
    y,
    metric_type='both'
)
df_adv_deb_w_p = df.copy()
df_adv_deb_w_p

## **7. Results comparison**
Now that we could observe how is the implementation of the mitigator in the model, we will compare the results between the baseline and the implementations with the mitigator to analyse how the metrics have changed.

In [ ]:
result = pd.concat([df_baseline, df_adv_deb, df_adv_deb_w_p], axis=1).iloc[:, [0,2,4,1]]
result.columns = ['Baseline','Mitigator without pipeline',"Mitigator with pipeline", 'Reference']
result

From the previous chart, we can see that although the actual metrics are still far from the ideal values, an improvement is obtained by applying this method in the dataset, compared with our baseline. 

Notice that we can analyse how these values are affected by varying the hyperparameters of the method. For example, this method allows us to set different hyperparameters such us the hidden size, the batch size, the adversarial loss weight and so on.

In this opportunity we will try with different values of the adversarial loss weight:

In [ ]:
def calculate_metrics(train_data, test_data, adversary_loss_weight, metrics_cols):
  inprocessing_model = AdversarialDebiasing(
        features_dim=train_data[0].shape[1],
        batch_size=32,
        hidden_size=64,
        adversary_loss_weight=adversary_loss_weight,
        verbose=1,
        use_debias=True,
        seed=42,
    ).transform_estimator()

  pipeline = Pipeline(
      steps=[
          ('scalar', StandardScaler()),
          ("bm_inprocessing", inprocessing_model),
      ]
  )

  X, y, group_a, group_b = train_data
  fit_params = {
      "bm__group_a": group_a, 
      "bm__group_b": group_b
  }

  pipeline.fit(X, y, **fit_params)

  X, y, group_a, group_b = test_data
  predict_params = {
      "bm__group_a": group_a,
      "bm__group_b": group_b,
  }
  y_pred = pipeline.predict(X, **predict_params)
  df = classification_bias_metrics(
      group_a,
      group_b,
      y_pred,
      y,
      metric_type='both'
  )
  # Calculate metrics
  metrics = [
      adversary_loss_weight, 
  ]
  metrics.extend([df['Value'].loc[col] for col in metrics_cols])
  return metrics

In [ ]:
metrics_val = []
metrics_cols = ['Statistical Parity', 'Disparate Impact', 'Four Fifths Rule']

for adversary_loss_weight in np.linspace(0, 1, 5):
  metrics_val.append(calculate_metrics(train_data, test_data, adversary_loss_weight, metrics_cols))

metrics = ['adversary_loss_weight']
metrics.extend(metrics_cols)
df_metrics = pd.DataFrame(metrics_val, columns=metrics)
df_metrics

In [ ]:
x_axis = df_metrics.adversary_loss_weight
plt.figure(figsize=(25,8))
plt.subplot(121)
plt.plot(x_axis, df_metrics['Disparate Impact'], 'o-', label='Disparate impact')
plt.plot(x_axis, np.ones_like(df_metrics['Disparate Impact']), label='Reference')
plt.legend(loc='upper left')
plt.xlabel('Adversary loss weight')
plt.subplot(122)
plt.plot(x_axis, df_metrics['Statistical Parity'], 'o-', label='Statistical Parity')
plt.plot(x_axis, np.zeros_like(df_metrics['Statistical Parity']), label='Reference')
plt.legend(loc='upper left')
plt.xlabel('Adversary loss weight')

From the previous graphs, we can see that increasing this value too much will not improve our model in terms of fairness, therefore we need to maintain this value as lower as possible. Let's try with a smaller range.

In [ ]:
metrics_val = []
metrics_cols = ['Statistical Parity', 'Disparate Impact', 'Four Fifths Rule']

for adversary_loss_weight in np.linspace(0, 0.3, 5):
  metrics_val.append(calculate_metrics(train_data, test_data, adversary_loss_weight, metrics_cols))

metrics = ['adversary_loss_weight']
metrics.extend(metrics_cols)
df_metrics = pd.DataFrame(metrics_val, columns=metrics)
df_metrics

In [ ]:
x_axis = df_metrics.adversary_loss_weight
plt.figure(figsize=(25,8))
plt.subplot(121)
plt.plot(x_axis, df_metrics['Disparate Impact'], 'o-', label='Disparate impact')
plt.plot(x_axis, np.ones_like(df_metrics['Disparate Impact']), label='Reference')
plt.legend(loc='upper left')
plt.xlabel('Adversary loss weight')
plt.subplot(122)
plt.plot(x_axis, df_metrics['Statistical Parity'], 'o-', label='Statistical Parity')
plt.plot(x_axis, np.zeros_like(df_metrics['Statistical Parity']), label='Reference')
plt.legend(loc='upper left')
plt.xlabel('Adversary loss weight')

Now we can see that although we obtain a similar behaviour as before, for this case a fairer model is obtained with an adversary loss weight value close to 0.3. Thereby, the selection of the model parameters will depend on our main objective, whether we are looking for fairness or accuracy.

## **Summary**
Through this tutorial we could present to the user how the `holisticai` library can be easily used to measure the bias present in classification models by the application of the `classification_bias_metrics` function, which returns the calculated values for different metrics and their references respectively. 

In addition, we shown how to mitigate the bias by applying some methods, for this opportunity we applied the "Adversarial debiasing" technique to train a fairness model, this is a in-processing method that leads to a fair classifier as the predictions cannot carry any group discrimination information that the adversary can exploit.